In [ ]:
# 전처리 한 거 안한거 비교하는 코드인듯
import os
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO   # ultralytics 설치 필요
from sklearn.metrics import precision_score, recall_score

# 설정
MODEL_PATH = "runs/train/Finetuned_141/weights/best.pt"
IMG_DIR    = "tree/train"
LBL_DIR    = "tree/250630_split/labels/train"
SAMPLE_N   = 100
IOU_THRESH = 0.5

def preprocess_contour(img):
    gray  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    cnts, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    canvas = np.zeros_like(gray)
    cv2.drawContours(canvas, cnts, -1, 255, 1)
    thick  = cv2.dilate(canvas, np.ones((2,2), np.uint8), iterations=1)
    return cv2.cvtColor(cv2.bitwise_not(thick), cv2.COLOR_GRAY2BGR)

def preprocess_adaptive(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    th   = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        blockSize=11, C=2
    )
    thick = cv2.dilate(th, np.ones((2,2), np.uint8), iterations=1)
    return cv2.cvtColor(cv2.bitwise_not(thick), cv2.COLOR_GRAY2BGR)

# IoU 계산
def compute_iou(boxA, boxB):
    # boxA, boxB 는 (x1, y1, x2, y2) 형태의 튜플
    # 교차 영역 좌표 계산
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # 교차 영역 넓이
    interArea = max(0, xB - xA) * max(0, yB - yA)

    # 각 박스 면적
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # IoU = 교집합 / (합집합)
    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-6)
    return iou

# 모델 로드
model = YOLO(MODEL_PATH)

# 샘플링
images = [f for f in os.listdir(IMG_DIR) if f.lower().endswith((".jpg",".png"))]
np.random.seed(None)
sampled = np.random.choice(images, min(SAMPLE_N, len(images)), replace=False)

records = []
for fn in sampled:
    img = cv2.imread(os.path.join(IMG_DIR, fn))
    # GT 박스 로드
    gt = []
    lblfile = os.path.join(LBL_DIR, os.path.splitext(fn)[0] + ".txt")
    if os.path.exists(lblfile):
        for line in open(lblfile):
            c, x, y, w, h = map(float, line.split())
            H,W = img.shape[:2]
            x1 = int((x - w/2)*W); y1 = int((y - h/2)*H)
            x2 = int((x + w/2)*W); y2 = int((y + h/2)*H)
            gt.append((x1,y1,x2,y2))
    # 세 가지 버전
    for mode, proc in [("vanilla", lambda x: x),
                       ("contour", preprocess_contour),
                       ("adaptive", preprocess_adaptive)]:
        proc_img = proc(img.copy())
        res = model.predict(source=proc_img, conf=0.5, iou=0.45, verbose=False)[0]
        pred = [tuple(map(int, box[:4])) for box in res.boxes.xyxy.cpu().numpy()]
        # 매칭 기준
        y_true, y_pred = [], []
        for gt_box in gt:
            matched = any(compute_iou(gt_box, p)>=IOU_THRESH for p in pred)
            y_true.append(1)
            y_pred.append(1 if matched else 0)
        # 예측 중 false positive도 체크
        for p in pred:
            # GT와 매칭 안 된 pred에 대해
            if not any(compute_iou(p, g)>=IOU_THRESH for g in gt):
                y_true.append(0)
                y_pred.append(1)
        # precision/recall 계산
        if y_true:
            prec = precision_score(y_true, y_pred)
            rec  = recall_score(y_true, y_pred)
        else:
            prec = rec = np.nan
        records.append({"image":fn, "mode":mode, "precision":prec, "recall":rec})

df = pd.DataFrame(records)
summary = df.groupby("mode").mean()[["precision","recall"]]
print(summary)


세 개의 패널(원본, 54 모델, 428 모델)을 가로로 붙여서 하나의 창에 띄우고, 키 입력에 따라 종료(q), 저장(s), 다음 이미지(space)가 동작합니다.

In [13]:
import os
import cv2
import numpy as np
import random
from ultralytics import YOLO

# ──────────────────────────────────────
# 1) 설정
# ──────────────────────────────────────
MODEL54_PATH    = "runs/train/Initial_54/weights/best.pt"
MODEL428_PATH   = "runs/train/Finetuned_428/weights/best.pt"
IMG_DIR         = "tree/train"
OUT_DIR         = "Prediction/Comparison"
CONF_THRESH     = 0.5
IOU_THRESH      = 0.1

# 클래스별 색상 정의
CLASS_COLORS = {
    "tree":   ( 34,139, 34),
    "branch": (139, 69, 19),
    "root":   (128,  64,128),
    "crown":  (  0,165,255),
    "fruit":  (  0,215,255),
    "gnarl":  (199, 21,133),
}

os.makedirs(OUT_DIR, exist_ok=True)

# ──────────────────────────────────────
# 2) 모델 로드
# ──────────────────────────────────────
model54  = YOLO(MODEL54_PATH)
model428 = YOLO(MODEL428_PATH)
names54  = model54.names
names428 = model428.names

# ──────────────────────────────────────
# 3) 예측 및 박스 그리기 함수
# ──────────────────────────────────────
def predict_and_draw(model, names, img):
    res = model.predict(source=img, conf=CONF_THRESH, iou=IOU_THRESH, verbose=False)[0]
    drawn = img.copy()
    # conf 값도 함께 가져오기
    boxes = res.boxes.xyxy.cpu().numpy()
    classes = res.boxes.cls.cpu().numpy()
    confs = res.boxes.conf.cpu().numpy()
    for box, cls, conf in zip(boxes, classes, confs):
        x1, y1, x2, y2 = map(int, box[:4])
        label = names[int(cls)]
        text = f"{label} {conf:.2f}"
        color = CLASS_COLORS.get(label, (0,255,0))
        cv2.rectangle(drawn, (x1, y1), (x2, y2), color, 2)
        cv2.putText(drawn, text, (x1, y1+15),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    return drawn

# ──────────────────────────────────────
# 4) 랜덤 이미지 선택
# ──────────────────────────────────────
all_images = [f for f in os.listdir(IMG_DIR)
              if f.lower().endswith(('.jpg','.png'))]
if not all_images:
    raise RuntimeError(f"No images found in {IMG_DIR}")

def get_random_image():
    fn = random.choice(all_images)
    path = os.path.join(IMG_DIR, fn)
    img = cv2.imread(path)
    return fn, img

# ──────────────────────────────────────
# 5) 메인 루프
# ──────────────────────────────────────
window_name = "Model54 | Model428"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
# 창 크기를 1600x800으로 설정
cv2.resizeWindow(window_name, 1600, 800)

while True:
    fn, img = get_random_image()
    # 두 모델 예측
    img54  = predict_and_draw(model54, names54, img)
    img428 = predict_and_draw(model428, names428, img)
    # 두 장을 가로로 이어 붙이기 (높이에 맞춰 resize)
    h = img.shape[0]
    img54  = cv2.resize(img54,  (int(img54.shape[1] * h / img54.shape[0]),  h))
    img428 = cv2.resize(img428, (int(img428.shape[1] * h / img428.shape[0]), h))
    combined = np.concatenate([img54, img428], axis=1)

    cv2.imshow(window_name, combined)
    key = cv2.waitKey(0) & 0xFF

    if key == ord('q'):
        break
    elif key == ord(' '):  # space
        continue
    elif key == ord('s'):
        basename, _ = os.path.splitext(fn)
        out_fn = f"{basename}_{int(cv2.getTickCount())}.png"
        out_path = os.path.join(OUT_DIR, out_fn)
        cv2.imwrite(out_path, combined)
        print(f"Saved comparison to {out_path}")
        continue

cv2.destroyAllWindows()


Saved comparison to Prediction/Comparison\나무_12_남_00590_295547259111800.png
Saved comparison to Prediction/Comparison\나무_11_남_04753_295663838527100.png
Saved comparison to Prediction/Comparison\나무_12_남_10933_295698032217600.png
Saved comparison to Prediction/Comparison\나무_9_남_04655_295767022448300.png


runs/train 밑의 각 폴더들의 results.csv 들을 분석해서 비교하는 코드

In [4]:
import os
import pandas as pd

RUNS_DIR   = "runs/train"
OUTPUT_CSV = "performance_summary.csv"

rows = []
for exp in os.listdir(RUNS_DIR):
    results_file = os.path.join(RUNS_DIR, exp, "results.csv")
    if not os.path.isfile(results_file):
        continue

    df = pd.read_csv(results_file)
    last = df.iloc[-1]

    # 디렉터리명에서 이미지 수(예: Finetuned_428 → 428) 뽑기
    try:
        images = int(exp.split("_")[-1])
    except ValueError:
        images = None

    rows.append({
        "experiment": exp,
        "images":     images,
        "precision":  last["metrics/precision(B)"],
        "recall":     last["metrics/recall(B)"],
        "map50":      last["metrics/mAP50(B)"],
        "map50_95":   last["metrics/mAP50-95(B)"]
    })

summary_df = pd.DataFrame(rows)
summary_df = summary_df.sort_values(by="images")
summary_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"✔️ Saved summary to {OUTPUT_CSV}")


✔️ Saved summary to performance_summary.csv
